In [2]:
from numpy.core.numeric import tensordot
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import time
import mediapipe as mp
import math
import csv
import plotly.offline as po
#import plotly.graph_objs as go
import plotly.graph_objects as go
import pandas as pd

#po.init_notebook_mode(connected=True)

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# make dir
timestr = time.strftime("%y%m%d-%H%M%S")
dirname = "recordimages/%s" % timestr
os.makedirs(dirname)


# ストリーム(Depth/Color)の設定
w = 1280
h = 720
config = rs.config()

config.enable_stream(rs.stream.color, w, h, rs.format.bgr8, 30)

config.enable_stream(rs.stream.depth, w, h, rs.format.z16, 30)
#
#config.enable_stream(rs.stream.color, 640, 360, rs.format.bgr8, 30)
#
#config.enable_stream(rs.stream.depth, 640, 360, rs.format.z16, 30)

# ストリーミング開始
# pipeline = rs.pipeline()
# profile = pipeline.start(config)

# Alignオブジェクト生成
align_to = rs.stream.color
align = rs.align(align_to)


try:
    count = 0

    with open('%s.csv' % timestr, 'w', newline="") as f:
        writer = csv.writer(f)

        # CSVファイルの1行目を書き込み
        csv_header = ["FrameNo.", "label", "label_count", "1_x", "1_x_est", "1_x_err", "1_y", "1_y_est", "1_y_err", "1_z", "1_z_est", "1_z_err", "2_x", "2_x_est", "2_x_err", "2_y", "2_y_est", "2_y_err", "2_z", "2_z_est", "2_z_err", "upperarm01.L_x", "upperarm01.L_x_est", "upperarm01.L_x_err", "upperarm01.L_y", "upperarm01.L_y_est", "upperarm01.L_y_err", "upperarm01.L_z", "upperarm01.L_z_est", "upperarm01.L_z_err", "upperarm01.R_x", "upperarm01.R_x_est", "upperarm01.R_x_err", "upperarm01.R_y", "upperarm01.R_y_est", "upperarm01.R_y_err", "upperarm01.R_z", "upperarm01.R_z_est", "upperarm01.R_z_err", "LELB_x", "LELB_x_est", "LELB_x_err", "LELB_y", "LELB_y_est", "LELB_y_err", "LELB_z", "LELB_z_est", "LELB_z_err", "LIEL_x", "LIEL_x_est", "LIEL_x_err", "LIEL_y", "LIEL_y_est", "LIEL_y_err", "LIEL_z", "LIEL_z_est", "LIEL_z_err", "RELB_x", "RELB_x_est", "RELB_x_err", "RELB_y", "RELB_y_est", "RELB_y_err", "RELB_z", "RELB_z_est", "RELB_z_err", "RIEL_x", "RIEL_x_est", "RIEL_x_err", "RIEL_y", "RIEL_y_est", "RIEL_y_err", "RIEL_z", "RIEL_z_est", "RIEL_z_err", "LIWR_x", "LIWR_x_est", "LIWR_x_err", "LIWR_y", "LIWR_y_est", "LIWR_y_err", "LIWR_z", "LIWR_z_est", "LIWR_z_err", "LOWR_x", "LOWR_x_est", "LOWR_x_err", "LOWR_y", "LOWR_y_est", "LOWR_y_err", "LOWR_z", "LOWR_z_est", "LOWR_z_err", "_x", "_x_est", "_x_err", "_y", "_y_est", "_y_err", "_z", "_z_est",
                      "_z_err", "RIWR_x", "RIWR_x_est", "RIWR_x_err", "RIWR_y", "RIWR_y_est", "RIWR_y_err", "RIWR_z", "RIWR_z_est", "RIWR_z_err", "ROWR_x", "ROWR_x_est", "ROWR_x_err", "ROWR_y", "ROWR_y_est", "ROWR_y_err", "ROWR_z", "ROWR_z_est", "ROWR_z_err", "E_x", "E_x_est", "E_x_err", "E_y", "E_y_est", "E_y_err", "E_z", "E_z_est", "E_z_err", "5_x", "5_x_est", "5_x_err", "5_y", "5_y_est", "5_y_err", "5_z", "5_z_est", "5_z_err", "6_x", "6_x_est", "6_x_err", "6_y", "6_y_est", "6_y_err", "6_z", "6_z_est", "6_z_err", "7_x", "7_x_est", "7_x_err", "7_y", "7_y_est", "7_y_err", "7_z", "7_z_est", "7_z_err", "8_x", "8_x_est", "8_x_err", "8_y", "8_y_est", "8_y_err", "8_z", "8_z_est", "8_z_err", "9_x", "9_x_est", "9_x_err", "9_y", "9_y_est", "9_y_err", "9_z", "9_z_est", "9_z_err", "10_x", "10_x_est", "10_x_err", "10_y", "10_y_est", "10_y_err", "10_z", "10_z_est", "10_z_err", "11_x", "11_x_est", "11_x_err", "11_y", "11_y_est", "11_y_err", "11_z", "11_z_est", "11_z_err", "_x", "_x_est", "_x_err", "_y", "_y_est", "_y_err", "_z", "_z_est", "_z_err", "LHEE_x", "LHEE_x_est", "LHEE_x_err", "LHEE_y", "LHEE_y_est", "LHEE_y_err", "LHEE_z", "LHEE_z_est", "LHEE_z_err", "E_x", "E_x_est", "E_x_err", "E_y", "E_y_est", "E_y_err", "E_z", "E_z_est", "E_z_err", "RHEE_x", "RHEE_x_est", "RHEE_x_err", "RHEE_y", "RHEE_y_est", "RHEE_y_err", "RHEE_z", "RHEE_z_est", "RHEE_z_err"]
        writer.writerow(csv_header)
            #カメラ位置
        camera = dict(
            eye = dict(x = 3.5, y = 0.05, z = 0.05 )
        )

        #traceを作成
        data = []
        marker_x = [1, 0, 0]
        marker_y = [0, 2, 0]
        marker_z = [0, 0, 3]
        
        #マーカー
        data.append(go.Scatter3d(
            x=marker_x,
            y=marker_y,
            z=marker_z,
            mode='markers',
            marker=dict(
                color='rgb(100,100,200)',
                size=3,
                opacity=0.8
            )
        ))
        
#         layoutを作成
        
#         traceとlayoutからfigureを作成
        fig = go.Figure(data=data)
        
        fig.update_layout(
            height = 1000,
            width = 500,
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=0
            ),
            #カメラ位置
            scene_camera = camera,
            scene=dict(
                aspectmode = 'cube',
                #aspectratio = dict(x=1, y=1, z=1),
                xaxis = dict(range=[-10,10]),
                yaxis = dict(range=[-10,10]),
                zaxis = dict(range=[0,20])),
            
            #scene=dict(aspectmode='cube'),
            showlegend=False,
            )
        f2 = go.FigureWidget(fig)
        display(f2)
    
        
        print("ok1")
        #fig.show()


finally:
    print("ok")

    # ストリーミング停止
#     pipeline.stop()

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

ok1
ok


In [7]:
data[0]

Scatter3d({
    'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
    'mode': 'markers',
    'x': [1, 1, 0],
    'y': [0, 2, 0],
    'z': [0, 0, 3]
})

In [12]:
for i in range(20):
    if i%2 == 0:
        data[0]['x']=(1,0,0)
    else:
        data[0]['x']=(1,1,0)
        
    fig = go.Figure(data=data)
    #f2 = go.FigureWidget(fig)
    display(f2)

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

...

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …